In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:

# Dict API
result = rs.glm_dict(
    response='ClaimCount',
    terms={'DrivAge': {'type': 'bs'}, 'Region': {'type': 'target_encoding'}},
    data=data,
    family='poisson'
).fit()

# Access smooth term results
if result.has_smooth_terms():
    print(f"Total EDF: {result.total_edf:.2f}")
    print(f"GCV: {result.gcv:.4f}")
    for st in result.smooth_terms:
        print(f"  {st.variable}: EDF={st.edf:.2f}, lambda={st.lambda_:.2f}")

Total EDF: 10.66
GCV: 0.3136
  DrivAge: EDF=8.66, lambda=0.96


In [ ]:
print(result.summary())

In [ ]:
result.predict(holdout_data)

In [ ]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "BonusMalus": {"type": "linear"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=data,
    family="poisson",
    offset="Exposure",
).fit()


In [ ]:
print(result.summary())

In [ ]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [3]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

/home/ralph/rustystats/python/rustystats/diagnostics.py:4081: RuntimeWarning: Could not retrieve matrices for score tests: 'FastResult' object has no attribute 'design_matrix'. Score tests for unfitted factors will be skipped.
  warnings.warn(


ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ bs(DrivAge, df=5) + TE(Region)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=406439.0, total_actual=21408.0, total_predicted=21315.39, loss=0.314473, deviance=127814.01, log_likelihood=-84680.83, aic=169383.66, gini=0.1104, auc=0.5552, ae_ratio=1.0043, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=40643.0, actual=0.037054, predicted=0.037207, ae_ratio=0.9959), DecileMetrics(decile=2, n=40643, exposure=40643.0, actual=0.043304, predicted=0.042651, ae_ratio=1.0153), DecileMetrics(decile=3, n=40643, exposure=40643.0, actual=0.045051, predicted=0.045329, ae_ratio=0.9939), DecileMetrics(decile=4, n=40643, exposure=40643.0, actual=0.047142, predicted=0.047712, ae_ratio=0.9881), DecileMetrics(decile=5, n=40643, exposure=40643.0, actual=0.049184, 

In [ ]:
predictions = result.predict(holdout_data)

In [ ]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [ ]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [ ]:
predictions2 = loaded.predict(holdout_data)

In [ ]:
import numpy as np
print(np.array_equal(predictions2, predictions))

In [ ]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [ ]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)